<a href="https://colab.research.google.com/github/vengie/capstone_project/blob/main/etl_load_reference_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyodbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.5/343.5 kB 10.3 MB/s eta 0:00:00


In [ ]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.message import EmailMessage
from email import encoders
import smtplib
import os
import json
import requests
import io

#import needed libraries
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
# this is imported from config folder
#import config.email as eml


In [ ]:
#config_url = 'https://raw.githubusercontent.com/vengie/capstone_project/main/config/config.json'

#content = open(config_url)
content = open('sample_data/config.json')
config = json.load(content)
email = config['local']
password = config['pass']

sender = 'admin@localmail.com'

In [ ]:
def send_mail(to, subject, body):
    message = MIMEMultipart()
    message['Subject'] = subject
    message['From'] = 'admin@localmail.com'
    message['To'] = to

    body_content = body
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()

    server = smtplib.SMTP('localhost')
    server.login(email, password)
    server.sendmail(sender, to, msg_body)
    #
    server.quit()

In [ ]:
#get password from environmnet var
pwd = os.environ['PGPASS']
uid = os.environ['PGUID']
server = "localhost"
db = "AdventureWorks"
port = "5432"
dir = r'\\YourPC\Shared\Data\Accounts'
to = 'youremail@domain.com

In [ ]:
#extract data from sql server
def extract():
    try:
        # starting directory
        directory = dir
        # iterate over files in the directory
        for filename in os.listdir(directory):
            #get filename without ext
            file_wo_ext = os.path.splitext(filename)[0]
            # only process excel files
            if filename.endswith(".xlsx"):
                f = os.path.join(directory, filename)
                # checking if it is a file
                if os.path.isfile(f):
                    df = pd.read_excel(f)
                    # call to load
                    load(df, file_wo_ext)
    except Exception as e:
        eml.send_mail(to, "File Upload, Data extract error: ", f"Data extract error: File location {dir}" + str(e))
        print("Data extract error: " + str(e))

In [ ]:
#load data to postgres
def load(df, tbl):
    try:
        rows_imported = 0
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... ')
        # save df to postgres
        df.to_sql(f"stg_{tbl}", engine, if_exists='replace', index=False)
        rows_imported += len(df)
        # add elapsed time to final print out
        print("Data imported successful")
        eml.send_mail(to, "File Uploaded, Data load successful: ", "Data load notification for : " + f"stg_{tbl}")
    except Exception as e:
        eml.send_mail(to, "File Upload Data load error: ", f"Data extract error: File location {dir}"  + str(e))
        print("Data load error: " + str(e))

try:
    #call extract function
    df = extract()
except Exception as e:
    eml.send_mail(to, "File Upload, Data extract error: ", f"Function call to file mapping, Data extract error: File location {dir}" + str(e))
    print("Error while extracting data: " + str(e))